In [1]:
%autosave 2

Autosaving every 2 seconds


In [2]:
import torch

In [3]:
def sigmoid_activation(x):
    """
    Arguments:
    ----------
    x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

## For single layer

In [4]:
torch.manual_seed(7)

# features are 5 random normal variables
features = torch.randn((1, 5))
print('features:\n', features)
# Same as features' dimension
weights = torch.rand_like(features)
print('weights:\n', weights)
# Bias term
bias = torch.rand((1,1))
print('bias:\n', bias)

features:
 tensor([[-0.1468,  0.7861,  0.9468, -1.1143,  1.6908]])
weights:
 tensor([[0.2868, 0.2063, 0.4451, 0.3593, 0.7204]])
bias:
 tensor([[0.0731]])


In [5]:
wi_xi = torch.sum(features * weights)
# This can also be used
# wi_xi = (features * weights).sum()
y_hat = sigmoid_activation(wi_xi + bias)

In [6]:
y_hat

tensor([[0.8072]])

## For multiple layer

In [7]:
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 3))

# Define the size of each layer in our network
n_input = features.shape[1]     # Number of input units, must match number of input features
n_hidden = 2                    # Number of hidden units 
n_output = 1                    # Number of output units

# Weights for inputs to hidden layer
W1 = torch.randn(n_input, n_hidden)
# Weights for hidden layer to output layer
W2 = torch.randn(n_hidden, n_output)

# and bias terms for hidden and output layers
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))